In [86]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 100

In [87]:
data = pd.read_csv('C:/Users/Игорь/Desktop/input/train.csv')
test = pd.read_csv('C:/Users/Игорь/Desktop/input/test.csv')

In [88]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

### Предварительная обработка данных, убираю пустые значения, корректирую выбросы

In [89]:
def check_data(df):
    df.loc[df['Square']<15, 'Square']=df['Square']*10
    df.loc[df['Square']>300,'Square']=df['Square']/10
    df.loc[(df['Rooms']<1) | (df['Rooms']>5),'Rooms']=np.round(df['Square']*0.6/20)
    df.loc[((df['LifeSquare']/df['Square'])<0.35) | ((df['LifeSquare']/df['Square'])>0.9), 'LifeSquare']=df['Square']*0.6
    df['LifeSquare']=df['LifeSquare'].fillna(df['Square']*0.6)
    df.loc[((df['KitchenSquare']/df['Square'])<0.10) | ((df['LifeSquare']/df['Square'])>0.2), 'KitchenSquare']=df['Square']*0.12
    df.loc[df['HouseYear']==4968, 'HouseYear']=1968
    df.loc[df['HouseYear']==20052011, 'HouseYear']=2011
    df.loc[(df['HouseFloor']==0) & (df['Floor']<12), 'HouseFloor']=12
    df.loc[(df['HouseFloor']==0) & (df['Floor']<17), 'HouseFloor']=17
    df.loc[(df['HouseFloor']==0) & (df['Floor']<30), 'HouseFloor']=30
    df.loc[df['HouseFloor']<df['Floor'], 'HouseFloor']=df['Floor']
    df.loc[(df['HouseFloor']>48) & (df['HouseYear']<2000),'HouseFloor']=np.round(df['HouseFloor']/10)
    df['Ecology_2']=(df['Ecology_2']=='B').astype(int)
    df['Ecology_3']=(df['Ecology_3']=='B').astype(int)
    df['Shops_2']=(df['Shops_2']=='B').astype(int)
    df=df.drop('Healthcare_1', axis=1)
    return df

In [90]:
train=check_data(train)
valid=check_data(valid)

In [91]:
train.info(), valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 9069 to 7270
Data columns (total 19 columns):
Id               7000 non-null int64
DistrictId       7000 non-null int64
Rooms            7000 non-null float64
Square           7000 non-null float64
LifeSquare       7000 non-null float64
KitchenSquare    7000 non-null float64
Floor            7000 non-null int64
HouseFloor       7000 non-null float64
HouseYear        7000 non-null int64
Ecology_1        7000 non-null float64
Ecology_2        7000 non-null int32
Ecology_3        7000 non-null int32
Social_1         7000 non-null int64
Social_2         7000 non-null int64
Social_3         7000 non-null int64
Helthcare_2      7000 non-null int64
Shops_1          7000 non-null int64
Shops_2          7000 non-null int32
Price            7000 non-null float64
dtypes: float64(7), int32(3), int64(9)
memory usage: 1.3 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 6252 to 4705
Data columns (total 19 columns):
Id 

(None, None)

### Рассчет средней цены квадратного метра в зависимости от района и инфраструктуры

Создаю df с признаками, которые относятся к характеристикам района и имеют наибольший коэффициент корреляции с DistrictId

In [92]:
train_dist=train[['DistrictId', 'Social_1', 'Social_2', 'Helthcare_2', 'Shops_1', 'Shops_2']]
train_dist.corr()

,DistrictId,Social_1,Social_2,Helthcare_2,Shops_1,Shops_2
DistrictId,1.000000,0.242861,0.162288,0.304945,0.173713,-0.025469
Social_1,0.242861,1.000000,0.955630,0.523949,0.524267,0.139777
Social_2,0.162288,0.955630,1.000000,0.474531,0.557988,0.140520
Helthcare_2,0.304945,0.523949,0.474531,1.000000,0.474736,0.119370
Shops_1,0.173713,0.524267,0.557988,0.474736,1.000000,0.135256
Shops_2,-0.025469,0.139777,0.140520,0.119370,0.135256,1.000000


Рассчитываю среднюю стоимость квадратного метра с учетом района и признаков, которые характеризуют район

In [93]:
train_dist['M2_price']=train['Price']/train['Square']
M2_price=train_dist.groupby(['DistrictId', 'Social_1', 'Social_2', 'Helthcare_2', 'Shops_1', 'Shops_2'], as_index=False)[['M2_price']].mean()
M2_price.head(2)

,DistrictId,Social_1,Social_2,Helthcare_2,Shops_1,Shops_2,M2_price
0,0,25,4756,5,8,1,4201.442477
1,0,30,5285,6,6,1,2589.743001


Рассчитываю среднюю стоимость квадратного метра с учетом района, но без учета признаков

In [94]:
M2_dist=train_dist.groupby(['DistrictId'], as_index=False)[['M2_price']].mean().rename(columns={'M2_price':'M2_dist'})
M2_dist.head(2)

,DistrictId,M2_dist
0,0,3537.801516
1,1,3211.478078


Средняя стоимость квадратного метра по всей выборке train или другими словами глобальную

In [95]:
M2_all=train_dist['M2_price'].mean()
M2_all

3897.987066279172

Создаю функцию для рассчета средней стоимости квадратного метра, произвожу с её помощью преобразование train и valid

In [96]:
def add_M2_price(df, M2_price=M2_price, M2_dist=M2_dist):
    global M2_all
    df = pd.merge(df, M2_dist, on=['DistrictId'], how='left')
    df['M2_dist'] = df['M2_dist'].fillna(M2_all)
    df = pd.merge(df, M2_price, on=['DistrictId', 'Social_1', 'Social_2', 'Helthcare_2', 'Shops_1', 'Shops_2'], how='left')
    df['M2_price'] = df['M2_price'].fillna(df['M2_dist'])
    df=df.drop('M2_dist', axis=1)
    return df

In [97]:
train=add_M2_price(train)
valid=add_M2_price(valid)

In [98]:
train.head(2)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,M2_price
0,14604,23,1.0,41.681380,22.796166,5.001766,14,17.0,2015,0.075779,1,1,6,1437,3,0,2,1,88504.384965,2561.348604
1,5621,23,3.0,163.495333,98.097200,19.619440,5,5.0,1977,0.014073,1,1,2,475,0,0,0,1,207007.956663,1983.515133


In [99]:
valid.head(2)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,M2_price
0,3702,21,2.0,48.812195,34.679779,5.857463,5,5.0,1963,0.034331,1,1,25,5487,3,0,4,1,181530.459031,4320.183949
1,12848,61,3.0,81.103039,49.310278,9.732365,4,4.0,1960,0.298205,1,1,16,4048,3,1,3,1,260456.004692,3892.224255


In [100]:
feats=['DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2',
       'Shops_1', 'Shops_2', 'M2_price']

In [101]:
from sklearn.ensemble import RandomForestRegressor as RFR
model = RFR(max_depth=8, random_state=42)
model.fit(train.loc[:, feats], train['Price'])
y_pred_train = model.predict(train.loc[:, feats])
from sklearn.metrics import r2_score as r2
r2(train['Price'], y_pred_train)

0.85569444220989

In [102]:
y_pred_valid = model.predict(valid.loc[:, feats])
r2(valid['Price'], y_pred_valid)

0.7082346811293292

### Test

In [103]:
test=check_data(test)

In [104]:
test=add_M2_price(test)

In [105]:
test.head(2)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,M2_price
0,725,58,2.0,49.882643,33.432782,5.985917,6,14.0,1972,0.310199,1,1,11,2748,1,0,0,1,3277.432032
1,15856,74,2.0,69.263183,41.557910,8.311582,6,6.0,1977,0.075779,1,1,6,1437,3,0,2,1,3254.720558


In [106]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null int32
Ecology_3        5000 non-null int32
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null int32
M2_price         5000 non-null float64
dtypes: float64(7), int32(3), int64(9)
memory usage: 722.7 KB


In [107]:
test['Price'] = model.predict(test.loc[:, feats])

In [108]:
test.loc[:,['Id','Price']].head()

,Id,Price
0,725,169070.223895
1,15856,221301.748594
2,5480,490363.627711
3,15664,327125.848122
4,14275,144716.442611


In [109]:
test.loc[:,['Id','Price']].to_csv('C:/Users/Игорь/Desktop/input/ITribul_predictions.csv', index=None)